In [ ]:
import tensorflow as tf
tf.config.set_visible_devices([], 'GPU')

from plotly import express as px
from typing import List, Tuple
import pickle
import numpy as np
from utils import multi_exp_data_to_fig

In [ ]:
import sys
sys.path.append('../../../../')

## load

In [ ]:
evaluation_type = 'validation'  # can be train / validation / test

In [ ]:
def get_max_ubiquitin_baselines():
    with open('/home/iscb/wolfson/doririmon/home/order/ubinet/repo/ubinet/datasets/patch_to_score/data_for_training/03_04_with_pesto/folds_training_dicts.pkl', 'rb') as f:
        base_folds_training_dicts = pickle.load(f)
        
    with open('/home/iscb/wolfson/doririmon/home/order/ubinet/repo/ubinet/datasets/patch_to_score/data_for_training/21_07_with_pesto/folds_training_dicts.pkl', 'rb') as f:
        ar_folds_training_dicts = pickle.load(f)
        
    model_to_training_dicts = {
        'base_max_ubiquitin': base_folds_training_dicts,
        'autoregressive_max_ubiquitin': ar_folds_training_dicts
    }

    multi_exp_data = dict()
    for model, training_dicts in model_to_training_dicts.items():
        exp_data = []
        for fold_index in range(len(base_folds_training_dicts)):
            fold = training_dicts[fold_index]
            ubiquitin_scores = fold[f'components_{evaluation_type}'][:, :, 1].numpy()
            ubiquitin_scores = ubiquitin_scores.max(axis=1) 
            labels = fold[f'labels_{evaluation_type}'].numpy()
            exp_data.append(np.stack((labels, ubiquitin_scores)))
        multi_exp_data[model] = exp_data
    
    return multi_exp_data

In [ ]:
multi_exp_data = get_max_ubiquitin_baselines()

In [ ]:
fig, exp_to_auc = multi_exp_data_to_fig(multi_exp_data, 
                            show_std_in_legend=False, log_scale=True, resolution=1_000)
fig.show()